In [24]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [15]:
class NoisyBatchNormalization(tf.keras.layers.BatchNormalization):
    
    def __init__(self, *args, **kwargs):
        tf.keras.layers.BatchNormalization.__init__(self, *args, **kwargs)
    
    def call(self, inputs, training, alpha, take_log=False):
        input_shape = K.int_shape(inputs)
        # Prepare broadcasting shape.
        ndim = len(input_shape)
        reduction_axes = list(range(len(input_shape)))
        del reduction_axes[self.axis]
        
        mean, var = tf.nn.moments(inputs,axes=reduction_axes)
        
        def broadcast_mean_var(x, ndim):
            if ndim == 2:
                return tf.reshape(x, shape=[1,-1])
            elif ndim == 4:
                return tf.reshape(x, shape=[1,1,1,-1])
            else:
                raise NotImplementedError
                
        if training:
            mean = broadcast_mean_var(mean, ndim)
            var = broadcast_mean_var(var, ndim)
            self.moving_mean = momentum * self.moving_mean + (1 - momentum) * mean
            self.var = momentum * self.moving_var + (1 - momentum) * var
        else:
            mean, var = self.moving_mean, self.moving_var
        
        norm = (inputs - mean) / tf.sqrt(var + self.epsilon)
        noise = tf.random.normal(input_shape)
        norm_noise = norm + alpha * noise
        
        
        
#         self.add_update([K.moving_average_update(self.moving_mean,
#                                                  mean,
#                                                  self.momentum),
#                          K.moving_average_update(self.moving_variance,
#                                                  var,
#                                                  self.momentum)],
#                         inputs)
        
        return self.gamma * norm_noise + self.beta
                

In [20]:
with tf.GradientTape() as tape:
    a = tf.constant([1,2,3])
    b = tf.Variable([4,5,6])
    c = a + b ** 2
    

In [21]:
print(tape.gradient(c, [b]))

[None]


In [25]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
tf.keras.layers.BatchNormalization()